In [ ]:
import datetime as dt
from io import StringIO
import psycopg2
import numpy as np
import numpy.matlib as matlib
import pandas as pd
import sqlalchemy as sa
from PyFin.api import *

In [ ]:
columns = ['trade_time', 'code', 'open', 'high', 'low', 'close', 'volume', 'amount']
codes = list(range(1, 3001))
dates = pd.to_datetime(bizDatesList('china.sse', '2010-01-01', '2018-04-20'))
time_stamps = pd.timedelta_range(start='09:30:00', end='11:30:00', freq='5min', closed='right').append(
                         pd.timedelta_range(start='13:00:00', end='15:00:00', freq='5min', closed='right'))

In [ ]:
def insert_date(date):
    values = np.random.randn(len(time_stamps) * len(codes), len(columns) - 2)
    time_stamp_col = matlib.repmat(date + time_stamps, 1, len(codes))[0]
    codes_column = np.repeat(codes, len(time_stamps))

    df = pd.DataFrame()
    df['trade_time'] = time_stamp_col
    df['code'] = codes_column
    for i, name in enumerate(columns[2:]):
        df[name] = values[:, i]
        
    df = df[columns]

    with StringIO() as sio:
        sio.write(df.to_csv(index=None, header=None, float_format='%.8f'))
        sio.seek(0)

        with psycopg2.connect("dbname=test user=postgres password=we083826 host=localhost") as con:
            with con.cursor() as c:
                c.copy_from(sio, "public.market_5min", columns=df.columns, sep=',')
                con.commit()
                
    print(f"{date} is finished at: {dt.datetime.now()}")

In [ ]:
from multiprocessing import Pool
pool = Pool(8)

In [ ]:
res = pool.map(insert_date, dates)